# Day 5 - Solution

## [Problem statement](https://adventofcode.com/2023/day/5)

In [1]:
"""
This module contains the solution to the Day 5 Advent of Code challenge.
"""
import os
import time

### Part 1

In [2]:
def lowest_location(seeds,dict_maps):
    """
    Calculates the lowest location among the locations of all seeds.

    Parameters:
        seeds (List[int]): List of seed numbers.
        dict_maps (dict): Dictionary mapping categories to lists of lists. 
                          Each list contains (destination, start, range).

    Returns:
        int: The lowest location found.
    """
    min_dest = float('inf')
    for seed in seeds:
        seed_input = seed
        for _, list_value in dict_maps.items():
            for value in list_value:
                dest = int(value[0])
                start = int(value[1])
                rang = int(value[2])
                if start <= seed_input < start + rang:
                    seed_input = dest + (seed_input - start)
                    break
        min_dest = min(min_dest, seed_input)
    return min_dest

For Part 2, I developed a working solution that took too much time to execute. In search of a more efficient solution, I found the following video on Youtube (https://www.youtube.com/watch?v=NmxHw_bHhGM) and adapted their code to my solution.

### Part 2 (inefficient)

In [3]:
def check_seed(candidate, seeds):
    """
    Checks if the candidate falls within any range defined by the seeds list.
    
    Args:
    candidate (int): The candidate value to be checked.
    seeds (list): A list where every other element is a start value and
    the following element is the range.

    Returns:
    int/bool: The candidate value if it falls within any of the ranges; 
    otherwise, False.
    """
    for k, seed in enumerate(seeds[::2]):
        seed_range = seeds[2*k + 1]
        if seed <= candidate < seed + seed_range:
            return candidate
    return False

def check_range(candidate, list_values):
    """
    Translates the candidate through a series of mappings defined in list_values.
    
    Args:
    candidate (int): The candidate value to be translated.
    list_values (list): A list of tuples/lists, where each sublist contains
    three integers (dest, start, range).

    Returns:
    int: The translated candidate value.
    """
    for value in list_values:
        dest = int(value[0])
        start = int(value[1])
        rang = int(value[2])
        if dest <= candidate < dest + rang:
            return start + (candidate - dest)
    return candidate

def reverse_finding(dictionary, seeds, key_names, start):
    """
    Finds a candidate value that, after a series of translations, falls within
    a range defined by the seeds list.
    
    Args:
    dictionary (dict): A dictionary containing the mappings for translations.
    seeds (list): A list where every other element is a start value and 
    the following element is the range.
    key_names (list): A list of keys to access specific mappings in 
    the dictionary.

    Returns:
    int: The found candidate value.
    """
    found =  False
    l = start
    while not found:
        candidate = l
        key_names_lenght = len(key_names)
        j = 0
        while j < key_names_lenght:
            candidate = check_range(candidate, dictionary[key_names[j]])
            j += 1
        if j == len(key_names):
            candidate = check_seed(candidate,seeds)
            if candidate:
                found = True
                return l
        l += 1
    return l

### Part 2 (efficient)

In [4]:
def lowest_location_2(seed_ranges, dict_maps, list_maps):
    """
    Calculates the lowest location among the locations of all seed ranges.

    Parameters:
        seeds (List[int, int]): List of seed ranges.
        dict_maps (dict): Dictionary mapping categories to lists of lists. 
                          Each list contains (destination, start, range).
        list_maps (List[strings]): List with the names of the dict_maps keys to iterate

    Returns:
        int: The lowest location found.
    """
    seeds = seed_ranges
    for name in list_maps:
        list_values = dict_maps[name]
        outputs = []
        while len(seeds) > 0:
            s,e = seeds.pop()
            for dest, start, rang in list_values:
                dest, start, rang = int(dest), int(start), int(rang)
                ost = max(s, start)
                oe = min(e, start + rang)
                if ost < oe:
                    outputs.append((ost - start + dest, oe - start + dest))
                    if ost>s:
                        seeds.append((s,ost))
                    if e>oe:
                        seeds.append((oe,e))
                    break
            else:
                outputs.append((s,e))
        seeds = outputs
    return min(seeds)[0]

### Solution

In [5]:
#Solution

#Code to open the input file
path = os.getcwd()
FILE = "day5.txt"

with open(path + '/' + FILE, encoding = 'utf-8') as f:
    lines = f.read().strip()
    lines = lines.split('\n\n')

seeds_input = lines[0].split(": ")[1].split(" ")
seeds_input = [int(x) for x in seeds_input]

dict_maps_input = {}
for i in range(1,len(lines)):
    parts = lines[i].split(":\n")
    dict_maps_input[parts[0]] = [x.split(" ") for x in lines[i].split(":\n")[1].split("\n")]

seed_ranges_input = [(seeds_input[i],seeds_input[i] + seeds_input[i+1])
                     for i in range(0,len(seeds_input),2)]

list_maps_input = [
    "seed-to-soil map",
    "soil-to-fertilizer map",
    "fertilizer-to-water map",
    "water-to-light map",
    "light-to-temperature map",
    "temperature-to-humidity map",
    "humidity-to-location map"
]
#Part 1
start_time = time.time()
print("Solution part 1: ", lowest_location(seeds_input,
                                           dict_maps_input))
#Part 2
print("Solution part 2: ", lowest_location_2(seed_ranges_input,
                                             dict_maps_input,
                                             list_maps_input))
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Solution part 1:  261668924
Solution part 2:  24261545
Execution time: 0.006265163421630859 seconds


In [6]:
# Inefficient solution
start_time = time.time()
seeds_input = lines[0].split(": ")[1].split(" ")
seeds_input = [int(x) for x in seeds_input]

dict_maps_input = {}
for i in range(len(lines)-1,0,-1):
    parts = lines[i].split(":\n")
    dict_maps_input[parts[0]] = [x.split(" ")
                                 for x in lines[i].split(":\n")[1].split("\n")]

key_names_input = [str(key) for key in dict_maps_input]

print("Inneficient solution to part 2: ",
      reverse_finding(dict_maps_input,
                      seeds_input,
                      key_names_input,
                      start = 0))
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Inneficient solution to part 2:  24261545
Execution time: 658.9173324108124 seconds
